In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearnex import patch_sklearn
import warnings
warnings.filterwarnings("ignore")
patch_sklearn()
#from sklearnex import patch_sklearn
#patch_sklearn(global_patch=True)
#import sklearn

#load data sets
books=pd.read_csv('Books.csv',error_bad_lines=False)
users=pd.read_csv('Users.csv',error_bad_lines=False)
ratings=pd.read_csv('Ratings.csv',error_bad_lines=False)

print("Books Data : ", books.shape)
print("Users Data : ",users.shape)
print("Ratings Data :",ratings.shape)

<h2>Books dataset preprocessing<h2>

In [ ]:
print("Columns: ", list(books.columns))
books.head()


In [ ]:
#renaming columns
books.rename(columns={'Book-Title':'Title',
                      'Book-Author':'Author',
                      'Year-Of-Publication':'YOP'
                       },inplace=True)
#checking for null values
books.isnull().sum()

In [ ]:
books.loc[books['Author'].isnull(),:]

In [ ]:
books.at[187689,'Author']='Other'

In [ ]:
books.loc[books['Publisher'].isnull(),:]

In [ ]:
books.at[128890,'Publisher']='Other'
books.at[129037,'Publisher']='Other'

In [ ]:
#checking column Year of Publication
books['YOP'].unique()

In [ ]:
books[books['YOP']=='DK Publishing Inc']

In [ ]:
books.at[209538,'YOP']=2000
books.at[221678,'YOP']=2000
books.at[209538,'Author']='DK Publishing Inc'
books.at[221678,'Author']='DK Publishing Inc'

In [ ]:
books[books['YOP']=='Gallimard']

In [ ]:
books.at[220731,'YOP']=2003
books.at[220731,'Author']='Gallimard'

In [ ]:
books['YOP'].unique()

In [ ]:
books.loc[books['Author'] == 'Gallimard',:]

In [ ]:
books.loc[books['Author'] == 'DK Publishing Inc',:]

In [ ]:
books['YOP'] = books['YOP'].astype(int)

In [ ]:
print(sorted(list(books['YOP'].unique())))


In [ ]:
books.drop_duplicates(keep='last', inplace=True) 
books.reset_index(drop = True, inplace = True)

<h2>User dataset pre-processing<h2>

In [ ]:
print("Columns: ", list(users.columns))
users.head()

In [ ]:
users.isnull().sum()


In [ ]:
print(users.isna().sum()) 

In [ ]:
print(sorted(list(users['Age'].unique())))

In [ ]:
users.describe()

<p>As we can see , mean is around 84k but the max is around 278k and min is around 0. Thus , we can confirm there are outlines. We need to handle them<p>

In [ ]:
required = users[users['Age'] <= 80]
required = required[required['Age'] >= 10]

users.loc[users['Age'] > 80, 'Age'] = 80    #outliers with age grater than 80 are substituted with mean 
users.loc[users['Age'] < 10, 'Age'] = 80    #outliers with age less than 10 years are substitued with mean
users['Age'] = users['Age'].fillna(80)      #filling null values with mean
users['Age'] = users['Age'].astype(int)     #changing datatype to int

print(sorted(list(users['Age'].unique())))

In [ ]:
## Drop duplicate rows
users.drop_duplicates(keep='last', inplace=True)
users.reset_index(drop=True, inplace=True)

<h2>Rating dataset preprocessing<h2>

In [ ]:
print("Columns: ", list(users.columns))
ratings.head()

In [ ]:
ratings.isnull().sum() 

In [ ]:
## Drop duplicate rows
ratings.drop_duplicates(keep='last', inplace=True)
ratings.reset_index(drop=True, inplace=True)

In [ ]:
ratings.info()

In [ ]:
plt.figure(figsize=(5,3))
sns.countplot(x="Book-Rating", data=ratings)

<h3>Merge datasets<h3>

In [ ]:
%pastebin

In [ ]:
ratings_with_name = ratings.merge(books,on='ISBN')
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index

In [ ]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [ ]:
y = filtered_rating.groupby('Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [ ]:
final_ratings = filtered_rating[filtered_rating['Title'].isin(famous_books)]

In [ ]:
pt = final_ratings.pivot_table(index='Title',columns='User-ID',values='Book-Rating')

In [ ]:
pt.fillna(0,inplace=True)

In [ ]:
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores = cosine_similarity(pt)

In [ ]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Title')['Title'].values))
        item.extend(list(temp_df.drop_duplicates('Title')['Author'].values))
        item.extend(list(temp_df.drop_duplicates('Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [ ]:
recommend('1984')

In [ ]:
import pickle
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))